## Reflection 
using autogen agent reflection technique to ask questions about CA insurance law.

## Setup

In [5]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [5]:
task = '''
        How much time does an insurance company have to
        accept or deny an insurance claim in California? and what about extentions

        Make sure the response takes into account any recent
        changes to the law and is accurate.

        Also state the date of your last update.
       '''


## Create a writer agent

In [5]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a legal writer. You write accurate and concise "
        "responses on given topics for lawyers. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

/Users/ajay/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

In California, an insurance company generally has 40 days to accept or deny an insurance claim after receiving proof of the claim. This period can be extended by the insurance company for up to 30 days if certain conditions apply. These conditions include the need for more time to investigate the claim or if they are waiting for additional information from the policyholder.

Last updated: September 2021.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [5]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a legal critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [5]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        How much time does an insurance company have to
        accept or deny an insurance claim in California? and what about extentions

        Make sure the response takes into account any recent
        changes to the law and is accurate.

        Also state the date of your last update.
       

--------------------------------------------------------------------------------
Writer (to Critic):

In California, an insurance company generally has 40 days to accept or deny an insurance claim after receiving proof of the claim. This period can be extended by the insurance company for up to 30 days if certain conditions apply. These conditions include the need for more time to investigate the claim or if they are waiting for additional information from the policyholder.

Last updated: September 2021.

--------------------------------------------------------------------------------
Critic (to Writer):

Your response provides a clear and concise answer to the ques

## Nested chat

In [5]:
legal_update_reviewer = autogen.AssistantAgent(
    name="Legal update Reviewer",
    llm_config=llm_config,
    system_message="You are an Insurance law update reviewer "
         ", tasked with tracking changes in insurance law."
         "You alert the team to any updates that might affect "
         "the organization and provide summaries of these changes "
         "and their potential impacts."
         "You stay informed about the latest developments in insurance law. "
         "Begin the review by stating your role.",
)


In [5]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [5]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [5]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [5]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content.
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": legal_update_reviewer,
     "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer,
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
     "max_turns": 1},
]


In [5]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [5]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        How much time does an insurance company have to
        accept or deny an insurance claim in California? and what about extentions

        Make sure the response takes into account any recent
        changes to the law and is accurate.

        Also state the date of your last update.
       

--------------------------------------------------------------------------------
Writer (to Critic):

In California, an insurance company generally has 40 days to accept or deny an insurance claim after receiving proof of the claim. This period can be extended by the insurance company for up to 30 days if certain conditions apply. These conditions include the need for more time to investigate the claim or if they are waiting for additional information from the policyholder.

Last updated: September 2021.

--------------------------------------------------------------------------------

********************************************************************************


## Get the summary

In [5]:
print(res.summary)

In California, an insurance company typically has 40 days to accept or deny an insurance claim after receiving proof of the claim. This timeline may be extended by up to 30 days under certain circumstances, such as the need for further investigation or awaiting additional information from the policyholder.

It is crucial for insurance companies in California to adhere to these statutory timelines to comply with state regulations effectively. Staying informed about any changes or updates in insurance laws in California is essential for ensuring compliance with timelines and other claim processing requirements.

Regular monitoring of legislative and regulatory developments in California is recommended to maintain compliance and streamline claims handling processes.

Last updated: September 2021.
